In [1]:
%matplotlib inline

## Synthetic DATA


Transfer Learning tutorial
==========================
**Author**: `Sasank Chilamkurthy <https://chsasank.github.io>`_

In this tutorial, you will learn how to train your network using
transfer learning. You can read more about the transfer learning at `cs231n
notes <http://cs231n.github.io/transfer-learning/>`__

Quoting this notes,

    In practice, very few people train an entire Convolutional Network
    from scratch (with random initialization), because it is relatively
    rare to have a dataset of sufficient size. Instead, it is common to
    pretrain a ConvNet on a very large dataset (e.g. ImageNet, which
    contains 1.2 million images with 1000 categories), and then use the
    ConvNet either as an initialization or a fixed feature extractor for
    the task of interest.

These two major transfer learning scenarios looks as follows:

-  **Finetuning the convnet**: Instead of random initializaion, we
   initialize the network with a pretrained network, like the one that is
   trained on imagenet 1000 dataset. Rest of the training looks as
   usual.
-  **ConvNet as fixed feature extractor**: Here, we will freeze the weights
   for all of the network except that of the final fully connected
   layer. This last fully connected layer is replaced with a new one
   with random weights and only this layer is trained.




In [2]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torchsample
from torchsample import transforms as ts_transforms
import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import Image
from tensorboard import SummaryWriter
from datetime import datetime

from torchsample.transforms import RangeNorm

import functions.fine_tune as ft

plt.ion()   # interactive mode

/home/mtezcan/anaconda3/lib/python3.6/site-packages/torchsample-0.1.2-py3.6.egg/torchsample/datasets.py:16: UserWarning: Cant import nibabel.. Cant load brain images


Load Data
---------

We will use torchvision and torch.utils.data packages for loading the
data.

The problem we're going to solve today is to train a model to classify
**ants** and **bees**. We have about 120 training images each for ants and bees.
There are 75 validation images for each class. Usually, this is a very
small dataset to generalize upon, if trained from scratch. Since we
are using transfer learning, we should be able to generalize reasonably
well.

This dataset is a very small subset of imagenet.

.. Note ::
   Download the data from
   `here <https://download.pytorch.org/tutorial/hymenoptera_data.zip>`_
   and extract it to the current directory.



In [3]:
# Data augmentation and normalization for training 
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Scale(256),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        #ts_transforms.RandomRotate(30)
    ]),
    'val': transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
'''
data_transforms = {
    'train': transforms.Compose([
        transforms.Scale(256),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        ts_transforms.TypeCast('float'), 
        #ts_transforms.ChannelsFirst(),
        ts_transforms.RangeNorm(0,1),
        ts_transforms.RandomRotate(30),
        ts_transforms.RandomGamma(.2,1.8),
        ts_transforms.RandomSaturation(.5,.9),
        ts_transforms.RandomBrightness(-.2,.2)
    ]),
    'val': transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        ts_transforms.TypeCast('float'), 
        #ts_transforms.ChannelsFirst(),
        ts_transforms.RangeNorm(0,1)
    ]),
}
'''



rootdir='//media//mtezcan//New Volume/HoardingImages//_rated//'
valdir='//media//mtezcan//New Volume/HoardingImages//_val//House//LR'
subdirs=os.listdir(rootdir)
print(subdirs)
roomdirs=['//BR','//Kitchen','//LR']
data_dir = '..//Data_Sets//pruned//good'
dsets = {x: datasets.ImageFolder_mtezcan([os.path.join(data_dir, x)], data_transforms[x])
         for x in ['train', 'val']}

#dsets = {'train':datasets.ImageFolder_mtezcan([rootdir+subdir+room for subdir in subdirs
#                                               for room in roomdirs], data_transforms['train']),
#         'val':datasets.ImageFolder_mtezcan([valdir], data_transforms['val'])}
 

weights,wpc = ft.make_weights_for_balanced_classes(dsets['train'].imgs, len(dsets['train'].classes))  
print(wpc)
weights = torch.DoubleTensor(weights) 
#sampler = {'train':torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights)) ,
sampler = {'train':None,
           'val':None}
shuffler={'train':True,'val':False}
dset_loaders = {x:torch.utils.data.DataLoader(dsets[x], batch_size=4,shuffle=shuffler[x],sampler=sampler[x], num_workers=4)
                for x in ['train','val']}
dset_sizes = {x: len(dsets[x]) for x in ['train', 'val']}
dset_classes = dsets['train'].classes
print(dset_classes)
use_gpu = torch.cuda.is_available()

#use_gpu=False
if use_gpu:
    print('GPU is available')
else:
    print('!!!!! NO CUDA GPUS DETECTED')

['BasicHouse_2017-07-01-rated', 'BriansHouse_2017-06-30-rated', 'RuralHome_2017-06-30-rated', 'SmallApt_2017-06-29-rated']
768
768
329
329
[12.59016393442623, 7.757575757575758, 10.816901408450704, 13.963636363636363, 8.629213483146067, 7.603960396039604, 5.605839416058394, 8.533333333333333, 11.815384615384616]
['1', '2', '3', '4', '5', '6', '7', '8', '9']
GPU is available


Visualize a few images
^^^^^^^^^^^^^^^^^^^^^^
Let's visualize a few training images so as to understand the data
augmentations.



In [ ]:

inputs, classes = next(iter(dset_loaders['train']))
# Make a grid from batch
out = torchvision.utils.make_grid(inputs,nrow=4)
print('Size of the input tensors in one batch after grid is  '+str(out.size()))
ft.imshow(out, title=[dset_classes[x] for x in classes])

Finetuning the convnet
----------------------

Load a pretrained model and reset final fully connected layer.




In [ ]:
#TO-BE-DELETED
train = []
labels = []
for i in range(100):
    category = (np.random.choice([0, 1]), np.random.choice([0, 1]))
    if category == (1, 0):
        train.append([np.random.uniform(0.1, 1), 0])
        labels.append([1, 0, 1])
    if category == (0, 1):
        train.append([0, np.random.uniform(0.1, 1)])
        labels.append([0, 1, 0])
    if category == (0, 0):
        train.append([np.random.uniform(0.1, 1), np.random.uniform(0.1, 1)])
        labels.append([0, 0, 1])
        


inputs, label = next(iter(dset_loaders['train']))
labels=[]
print(label)
for lbl in label:
    x=np.zeros(11)
    x[lbl:lbl+3]=1
    x=x[1:-1]
    labels.append(x)
print(labels)
    
i=0
labelsv = Variable(torch.FloatTensor(labels).cuda()).view(-1, 9)
print(labelsv)
criterion = nn.MultiLabelSoftMarginLoss()


In [17]:
model_ft = models.resnet18(pretrained=True)
#model_ft=torch.load('./saved_models/resnet_synthetic_7_8_17.mdl')
#print('The CNN model is')
#print(model_ft)
#print(list(model_ft.children())[:-1])
num_ftrs = model_ft.fc.in_features
print('Number of features in the fine tune layer is '+str(num_ftrs))
#setattr(model_ft.classifier, '6', nn.Linear(num_ftrs, 9))
model_ft.fc = nn.Linear(num_ftrs, 9)
#print(model_ft)

if use_gpu:
    model_ft = model_ft.cuda()

#criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
#optimizer_ft = optim.Adam(model_ft.parameters())
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

Number of features in the fine tune layer is 512


In [4]:
model_ft = models.alexnet(pretrained=True)

#print('The CNN model is')

#print(list(model_ft.children())[:-1])
num_ftrs = model_ft.classifier[6].out_features
print('Number of features in the fine tune layer is '+str(num_ftrs))
setattr(model_ft.classifier, '7', nn.ReLU(inplace=True))
setattr(model_ft.classifier, '8', nn.Dropout())
setattr(model_ft.classifier, '9', nn.Linear(num_ftrs,9))
print(model_ft)
#model_ft.fc = nn.Linear(num_ftrs, 9)
#print(model_ft)

if use_gpu:
    model_ft = model_ft.cuda()

#criterion = nn.CrossEntropyLoss()
criterion = nn.MultiLabelSoftMarginLoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

Number of features in the fine tune layer is 1000
AlexNet (
  (features): Sequential (
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU (inplace)
    (2): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU (inplace)
    (5): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU (inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU (inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU (inplace)
    (12): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  )
  (classifier): Sequential (
    (0): Dropout (p = 0.5)
    (1): Linear (9216 -> 4096)
    (2): ReLU (inplace)
    (3): Dropout (p = 0.5)
    (4): Linear (4096 -> 4096)
    (5): ReLU (inplace)
    (6): Linear

In [18]:
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
import warnings
warnings.filterwarnings("ignore")
writer = SummaryWriter('runs/'+datetime.now().strftime('%B%d  %H:%M:%S'))
model_ft = ft.train_model(model_ft, criterion, optimizer_ft, ft.exp_lr_scheduler,dset_loaders,dset_sizes,writer,
                        use_gpu=use_gpu,num_epochs=50,batch_size=32,num_log=200)

Epoch 0/49
----------
LR is set to 0.001
train Loss: 0.1448 Acc: 0.2018 CIR-1: 0.5312
val Loss: 0.1223 Acc: 0.2827 CIR-1: 0.7143

Epoch 1/49
----------
train Loss: 0.1242 Acc: 0.2422 CIR-1: 0.6576
val Loss: 0.1127 Acc: 0.2766 CIR-1: 0.6960

Epoch 2/49
----------
train Loss: 0.1157 Acc: 0.2578 CIR-1: 0.7122
val Loss: 0.1047 Acc: 0.3283 CIR-1: 0.7599

Epoch 3/49
----------
train Loss: 0.1139 Acc: 0.2617 CIR-1: 0.7044
val Loss: 0.1035 Acc: 0.3343 CIR-1: 0.7599

Epoch 4/49
----------
train Loss: 0.1061 Acc: 0.3021 CIR-1: 0.7565
val Loss: 0.1002 Acc: 0.3374 CIR-1: 0.7842

Epoch 5/49
----------
train Loss: 0.1022 Acc: 0.3125 CIR-1: 0.7917
val Loss: 0.0964 Acc: 0.3951 CIR-1: 0.8207

Epoch 6/49
----------
train Loss: 0.0983 Acc: 0.2956 CIR-1: 0.7904
val Loss: 0.0952 Acc: 0.3678 CIR-1: 0.8328

Epoch 7/49
----------
LR is set to 0.0001
train Loss: 0.0936 Acc: 0.3398 CIR-1: 0.8503
val Loss: 0.0953 Acc: 0.3799 CIR-1: 0.8237

Epoch 8/49
----------
train Loss: 0.0939 Acc: 0.3490 CIR-1: 0.8372
val Lo

In [ ]:
torch.save(model_ft,'./saved_models/resnet_synthetic_7_8_17.mdl')

In [14]:
model_ft=torch.load('./saved_models/resnet_synthetic_7_8_17.mdl')
if use_gpu:
    model_ft = model_ft.cuda()

#criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
#optimizer_ft = optim.Adam(model_ft.parameters())
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
#criterion = nn.CrossEntropyLoss()
criterion = nn.MultiLabelSoftMarginLoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# --- to-be-optimized ---

In [ ]:
#print(next(iter(dset_loaders['train']))[0])
#model_ft = models.alexnet(pretrained=True)

model_ft=model_ft.cpu()
#new_classifier = nn.Sequential(*list(model_ft.classifier.children())[:-5])
#model_ft.classifier = new_classifier
#print(model_ft)
model_params= list(model_ft.children())
#model_params[1]=list(model_params[1])
#print(model_params)
new_ft = nn.Sequential(*list(model_params)[:-1])
#print(new_ft)

fvec_tr=np.zeros((20000,512))
label_tr=np.zeros((20000))

fvec_val=np.zeros((20000,512))
label_val=np.zeros((20000))
count=0;

#inputs_t, classes_t = data=next(iter(dset_loaders['train']))
#print(inputs_t.size())
#fvec_t=new_ft(Variable(inputs_t))

for data in dset_loaders['train']:
    inputs_t, classes_t = data
    fvec_t=new_ft(Variable(inputs_t))
    #print(fvec_t)
    fvec_t_cpu=fvec_t.cpu()
    if(fvec_t_cpu.data.numpy().shape[0]==4):
        fvec_tr[count:count+4,:]=fvec_t_cpu.data.numpy().reshape(4,-1)
        label_tr[count:count+4]=classes_t.short().numpy()
        count +=4
fvec_tr=fvec_tr[:count,:]
label_tr=label_tr[:count]


count=0;
for data in dset_loaders['val']:
    inputs_t, classes_t = data
    fvec_t=new_ft(Variable(inputs_t))
    fvec_t_cpu=fvec_t.cpu()
    if(fvec_t_cpu.data.numpy().shape[0]==4):
        fvec_val[count:count+4,:]=fvec_t_cpu.data.numpy().reshape(4,-1)
        label_val[count:count+4]=classes_t.short().numpy()
        count +=4
    
fvec_val=fvec_val[:count,:]
label_val=label_val[:count]

'''
print('The CNN model is')
print(model_ft)
num_ftrs = model_ft.classifier[6].in_features
print('Number of features in the fine tune layer is '+str(num_ftrs))
setattr(model_ft.classifier, '6', nn.Linear(num_ftrs, 9))
#model_ft.classifier['6'] = nn.Linear(num_ftrs, 2)
print(model_ft)
'''

print('Size of the input in training is '+str(fvec_tr.shape))
print('Size of the labels in training is '+str(label_tr.shape))

print('Size of the input in validation is '+str(fvec_val.shape))
print('Size of the labels in validation is '+str(label_val.shape))


In [ ]:
print(fvec_val.shape)
print(fvec_val[1,:])

In [ ]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC
#clf = SVC(C=1,kernel='linear',class_weight='balanced',decision_function_shape='ovo')
clf=OneVsOneClassifier(LinearSVC(C=1))
clf.fit(fvec_tr,label_tr)
label_pred=clf.predict(fvec_val)
abs_err=np.abs(label_pred-label_val)
cir1=np.mean(abs_err<=1)
print(cir1)

In [ ]:
from sklearn.svm import SVC

c_val=np.power(2,np.arange(0.,8.,1.))
g_val=np.power(2,np.arange(-8.,0.,1.))

#c_val=[1e-10]
#g_val=[np.power(2,-7.)]
cir1_max=0.
c_max=0
g_max=0
for c_1 in c_val:
    for g in g_val:
        
        clf_svc = SVC(C=c_1,kernel='rbf',gamma=g,shrinking=False,class_weight='balanced')
        clf=OneVsOneClassifier(clf_svc)
        #print(label_tr)
        clf.fit(fvec_tr,label_tr)

        label_pred=clf.predict(fvec_val)
        #print(label_pred)
        abs_err=np.abs(label_pred-label_val)
        #print(abs_err)
        cir1=np.mean(abs_err<=1)
        print('C = '+str(c_1),', g = ',str(g),', cir-1 = '+str(cir1))
        if(cir1>cir1_max):
            cir1_max=cir1
            c_max=c_1
            g_max=g
            

clf = SVC(C=c_max,kernel='linear',gamma=g_max)#,class_weight='balanced')
clf.fit(fvec_tr,label_tr)
label_pred=clf.predict(fvec_val)
abs_err=np.abs(label_pred-label_val)
print('Max CIR-1 achieved is '+str(np.mean(abs_err<=1)) +' with c='+str(c_max),', g='+str(g_max))
print('CIR-0 = '+str(np.mean(abs_err<=0)))
print('CIR-2 = '+str(np.mean(abs_err<=2)))
print(abs_err)
print(label_val)

In [ ]:
model_ft = models.resnet18(pretrained=True)

#print('The CNN model is')
#print(model_ft)
#print(list(model_ft.children())[:-1])
num_ftrs = model_ft.fc.in_features
print('Number of features in the fine tune layer is '+str(num_ftrs))
#setattr(model_ft.classifier, '6', nn.Linear(num_ftrs, 9))
model_ft.fc = nn.Linear(num_ftrs, 9)
#print(model_ft)

#print(next(iter(dset_loaders['train']))[0])
new_ft = nn.Sequential(*list(model_ft.children())[:-1])
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=4,
                                               shuffle=True, num_workers=4)
                for x in ['train', 'val']}

fvec_tr=np.zeros((X_tr.shape[0],512))
for k in range(0,10,10):#(0,inputs.shape[0],10):
    print(k)
    fvec_now=new_ft(Variable(torch.from_numpy(X_tr[k:k+10,:,:,:])))
    print(fvec_now.numpy())

print(fvec)


'''
print('The CNN model is')
print(model_ft)
num_ftrs = model_ft.classifier[6].in_features
print('Number of features in the fine tune layer is '+str(num_ftrs))
setattr(model_ft.classifier, '6', nn.Linear(num_ftrs, 9))
#model_ft.classifier['6'] = nn.Linear(num_ftrs, 2)
print(model_ft)
'''

if use_gpu:
    model_ft = model_ft.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

Train and evaluate
^^^^^^^^^^^^^^^^^^

It should take around 15-25 min on CPU. On GPU though, it takes less than a
minute.




In [ ]:
visualize_model(model_ft)

ConvNet as fixed feature extractor
----------------------------------

Here, we need to freeze all the network except the final layer. We need
to set ``requires_grad == False`` to freeze the parameters so that the
gradients are not computed in ``backward()``.

You can read more about this in the documentation
`here <http://pytorch.org/docs/notes/autograd.html#excluding-subgraphs-from-backward>`__.




In [ ]:
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    #print(param)
    param.requires_grad = False
    
for param in model_conv.fc.parameters():
    #print(param)
    param.requires_grad = True

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 9)

if use_gpu:
    model_conv = model_conv.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opoosed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

Train and evaluate
^^^^^^^^^^^^^^^^^^

On CPU this will take about half the time compared to previous scenario.
This is expected as gradients don't need to be computed for most of the
network. However, forward does need to be computed.




In [ ]:
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=25)

In [ ]:
visualize_model(model_conv)

plt.ioff()
plt.show()